In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
import os.path
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import accuracy_score, classification_report
import keras

Data augmentation

In [ ]:
train_data_dir = '../model/dataset/train'
val_data_dir = '../model/dataset/valid/'
test_data_dir = '../model/dataset/test/'

# Image size and batch size
img_size = (350, 350)
batch_size = 64

# Data augmentations performed on training set
train_datagen = ImageDataGenerator(
    rescale=1./255.,
    horizontal_flip=True,
)

train_images = train_datagen.flow_from_directory(
    train_data_dir,
    color_mode='rgb',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary'
    
)

# Data augmentation for the valdiation set
validation_datagen = ImageDataGenerator(
    rescale=1./255.,
    horizontal_flip=True
)

validation_images = validation_datagen.flow_from_directory(
    val_data_dir,
    color_mode='rgb',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary'
)

# Data augmentation performed on testing set
testing_datagen = ImageDataGenerator(
    rescale=1./255.,
)

testing_images = testing_datagen.flow_from_directory(
    test_data_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False
)

In [ ]:
plt.figure(figsize=(15, 15))
for images, labels in train_images:
    for i in range(12):
        ax = plt.subplot(3, 4, i + 1) 
        image = images[i]

        plt.imshow(image)
      
        plt.axis("off")
    break  # To show only the first batch
plt.show()  # To display the plot

In [ ]:
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dropout, Flatten, Dense, BatchNormalization
from keras.models import Sequential
from tensorflow.keras import layers, models


model = models.Sequential([
    layers.Conv2D(16, (3, 3), activation='relu', input_shape=(350, 350, 3), padding="same"),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(32, (3, 3), activation='relu',  padding="same"),
    layers.MaxPooling2D((2, 2)),
    
    layers.Conv2D(64, (3, 3), activation='relu',  padding="same"),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.15),
    
    #layers.Conv2D(128, (3, 3), activation='relu',  padding="same"),
    #layers.MaxPooling2D((2, 2)),
    #layers.Dropout(0.15),
  
    layers.Flatten(),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid') 
])


model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

# Train the model
history = model.fit(
    train_images,
    steps_per_epoch=train_images.samples // batch_size,
    epochs=14,
    validation_data=validation_images,
    
     callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=4,
            restore_best_weights=True
        ),
         tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            patience=3
        )
       
    ]
)

In [ ]:
result = model.evaluate(testing_images)

print("Test loss: {:.5f}".format(result[0]))
print("Test accuracy: {:.2f}%".format(result[1] * 100))


# Predict on the test set
predictions = model.predict(testing_images)

# Get predicted classes
predicted_classes = (predictions > 0.5).astype(int)  

# Get true classes
true_classes = testing_images.classes

# Calculate accuracy
accuracy = accuracy_score(true_classes, predicted_classes)
print(f'Test Accuracy: {accuracy}')

# Generate a classification report
class_report = classification_report(true_classes, predicted_classes)
print('Classification Report:\n', class_report)

Confusion matrix

In [ ]:
conf_matrix = confusion_matrix(true_classes, predicted_classes)

# Plot the confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues',
            xticklabels=testing_images.class_indices,
            yticklabels=testing_images.class_indices)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

In [ ]:
#Save the model 
model.save('../app/wildfiremodel.h5')